In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, DistilBertForSequenceClassification, AdamW
import torch
import json
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW

2023-06-13 10:48:22.684110: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 10:48:24.580716: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Step 1: Load and preprocess the dataset
def load_dataset(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

train_data = load_dataset('sqlData-train.json')
test_data = load_dataset('sqlData-test.json')

In [4]:
# Step 2: Tokenization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Step 3: Model Configuration
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.config.num_labels = tokenizer.vocab_size  # Set the number of labels to the tokenizer vocab size


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [5]:
# Step 4: Data Encoding
class SQLDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        question = item['question']
        sql = item['sql']
        encoded_inputs = tokenizer(question, truncation=True, padding='max_length')
        input_ids = encoded_inputs['input_ids']
        attention_mask = encoded_inputs['attention_mask']
        encoded_outputs = tokenizer(sql, truncation=True, padding='max_length')
        output_ids = encoded_outputs['input_ids']
        output_attention_mask = encoded_outputs['attention_mask']
        return {'input_ids': torch.tensor(input_ids),
                'attention_mask': torch.tensor(attention_mask),
                'output_ids': torch.tensor(output_ids),
                'output_attention_mask': torch.tensor(output_attention_mask)}

train_dataset = SQLDataset(train_data, tokenizer)
test_dataset = SQLDataset(test_data, tokenizer)

In [6]:
# Step 5: Data Loading
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [8]:
# Step 6: Model Training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

model.train()
for epoch in range(5):  # Adjust the number of epochs as needed
    total_loss = 0
    total_correct = 0
    total_samples = 0
    
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        output_ids = batch['output_ids'].to(device)
        output_attention_mask = batch['output_attention_mask'].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=output_ids)
        loss = outputs.loss
        total_loss += loss.item()
        
        logits = outputs.logits

        # Adjust the dimensions of the predicted output and the ground truth output
        logits = logits.view(-1, logits.shape[-1])
        output_ids = output_ids.view(-1)
        
        _, predictions = torch.max(logits, dim=1)
        
        total_correct += (predictions == output_ids).sum().item()
        total_samples += output_ids.numel()
        
        loss.backward()
        optimizer.step()
    
    avg_loss = total_loss / len(train_loader)
    accuracy = total_correct / total_samples
    print(f"Epoch {epoch+1}/{5}, Train Loss: {avg_loss}, Train Accuracy: {accuracy}")

ValueError: Expected input batch_size (8) to match target batch_size (4096).